# Introduction

## Business Question

---

> **Guiding question:** What are the top 5 zip codes based on the ROI for the cit of Pittsburgh?
>
>
> **Evaluation Metric:** ROI/Risk
>
>
> **Dataset:** Zillow data from 1996-2018
>
>
> **Goal:** time series modeling for each zip code to calculate forecasted sale prices.
>
>
> 

---

# **Workflow: Start -> Finish**

## Import

In [ ]:
# import warnings
# warnings.filterwarnings('ignore')

## Data Handling
import pandas as pd
import numpy as np

## Visualizations
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels
import statsmodels.tsa.api as tsa
from statsmodels.tsa.seasonal import seasonal_decompose

import pmdarima as pmd

## Settings
%matplotlib inline
plt.rcParams['figure.figsize']=(12,6)
plt.style.use('seaborn-talk')
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')
pd.set_option('max_rows', 100)

from bmc_functions import eda

In [ ]:
%load_ext autoreload
%autoreload 2

## Reading Data

In [ ]:
## Reading data
source = '../data/zillow_data.csv'
data = pd.read_csv(source)
data.head(5)

In [ ]:
## Initial inspection
data.info()

## Creating Subset of Zipcodes

---

> The dataset is much larger than I need for my purposes, so I will select only the zip codes for the Pittsburgh Metro area.
>
>
> To select this data, I will filter the initial dataframe by selecting "Pittsburgh" from the "city" column.

---

In [ ]:
## Selecting the city of Pittsburgh 

pitt_df = data[data['City'] == 'Pittsburgh']
pitt_df

In [ ]:
## Examining Statistics for the city of Pittsburgh 
eda.report_df(pitt_df).T

In [ ]:
# ## Inspecting overall data for CA - transposed and dropping RegionID, SizeRank

# pitt_zips = pitt_df.pivot_table(index= 'RegionName').T[:-2]
# pitt_zips

# Data Preprocessing

In [ ]:
# def get_datetimes(df):
#     """
#     Takes a dataframe:
#     returns only those column names that can be converted into datetime objects 
#     as datetime objects.
#     NOTE number of returned columns may not match total number of columns in passed dataframe
#     """
    
#     return pd.to_datetime(df.columns.values[7:], format='%Y-%m')

In [ ]:
# pitt_df.columns.values[7:] = get_datetimes(pitt_df)
# pitt_df

# Melting DataFrame

In [ ]:
def melt_data(df):
    """
    Takes the zillow_data dataset in wide form or a subset of the zillow_dataset.  
    Returns a long-form datetime dataframe with the datetime column names
    as the index and the values as the 'values' column.
    
    If more than one row is passes in the wide-form dataset, the values column
    will be the mean of the values from the datetime columns in all of the rows.
    """
    
    melted = pd.melt(df, id_vars=['RegionName', 'RegionID', 'SizeRank',
                                  'City', 'State', 'Metro', 'CountyName'],
                     var_name='time')
    melted['time'] = pd.to_datetime(melted['time'], infer_datetime_format=True)
    melted = melted.dropna(subset=['value'])
    return melted

In [ ]:
pitt_melted = melt_data(pitt_df)
pitt_melted

## T/T Split

In [1]:
def ts_spilt(timeseries_df, threshold):
    tts_cutoff = round(ts.shape[0]*threshold)
    train = ts.iloc[:tts_cutoff]
    test = ts.iloc[tts_cutoff:]

    ## Plot
    ax = train.plot(label='Train')
    test.plot(label='Test')
    ax.legend()
    
    return train, test

In [4]:
## Creating the train/test split datasets

ts_split(pitt_df, .8)

NameError: name 'ts_split' is not defined

# ❌ TODO: manually code, then  functionize

## Stationarity Check (Tr)

### Dickey Fuller Test for Stationarity

In [ ]:
## ADFuller test and results

def adfuller_test_df(ts,index=['AD Fuller Results']):
    """Returns the AD Fuller Test Results and p-values for the null hypothesis
    that there the data is non-stationary (that there is a unit root in the data)"""
    
    df_res = tsa.stattools.adfuller(ts)
    
    names = ['Test Statistic','p-value','#Lags Used','# of Observations Used']
    res  = dict(zip(names,df_res[:4]))
    
    res['p<.05'] = res['p-value']<.05
    res['Stationary?'] = res['p<.05']
    
    if isinstance(index,str):
        index = [index]
    res_df = pd.DataFrame(res,index=index)
    res_df = res_df[['Test Statistic','#Lags Used',
                     '# of Observations Used','p-value','p<.05',
                    'Stationary?']]
    return res_df

In [ ]:
def stationarity_check(TS,window=8,plot=True,index=['AD Fuller Results']):
    """Adapted from https://github.com/learn-co-curriculum/dsc-removing-trends-lab/tree/solution"""
    
    # Calculate rolling statistics
    roll_mean = TS.rolling(window=window, center=False).mean()
    roll_std = TS.rolling(window=window, center=False).std()
    
    # Perform the Dickey Fuller Test
    dftest = adfuller_test_df(TS,index=index)
    
    if plot:
        # Plot rolling statistics:
        fig = plt.figure(figsize=(12,6))
        plt.plot(TS, color='blue',label=f'Original (freq={TS.index.freq})')
        plt.plot(roll_mean, color='red', label=f'Rolling Mean (window={window})')
        plt.plot(roll_std, color='black', label = f'Rolling Std (window={window})')
        plt.legend(loc='best')
        plt.title('Rolling Mean & Standard Deviation')
        display(dftest)
        plt.show(block=False)
        
    return dftest

### Dickey Fuller Test

In [ ]:
df_res = tsa.stattools.adfuller(ts)

In [ ]:
names = ['Test Statistic','p-value','#Lags Used','# of Observations Used']
res  = dict(zip(names,df_res[:4]))

res['p<.05'] = res['p-value']<.05
res['Stationary?'] = res['p<.05']

if isinstance(index,str):
    index = [index]
res_df = pd.DataFrame(res,index=index)
res_df = res_df[['Test Statistic','#Lags Used',
                 '# of Observations Used','p-value','p<.05',
                'Stationary?']]

### Removing Trends, Seasonality

In [ ]:
## Testing on differenced data
tz_diff = test_zip.diff().dropna()
tz_diff.plot()
adfuller_test_df(tz_diff)

In [ ]:
## Log Transform, plot and get adfuller test
tz_log = np.log(test_zip)
tz_log.plot()
adfuller_test_df(tz_log)

In [ ]:
## Subtract Rolling mean
tz_rm = test_zip - test_zip.rolling(window=4).mean()
tz_rm.dropna(inplace=True)
tz_rm.plot()
adfuller_test_df(tz_rm)

In [ ]:
## Exponentially-Weighted Mean
tz_ewm = test_zip-test_zip.ewm(4).mean()
tz_ewm.dropna(inplace=True)
tz_ewm.plot()
adfuller_test_df(tz_ewm)

In [ ]:
## Seasonal Decomposition
decomp = seasonal_decompose(test_zip)
decomp.plot();

In [ ]:
decomp.seasonal

In [ ]:
## Save seasonal/trend/resid in a dictionary.

decomp_dict = {'seasonal': decomp.seasonal,
              "trend": decomp.trend,
              'residuals': decomp.resid}

decomp_dict

In [ ]:
## Make a list of adfuller results to append
results = []
## Save results of orig ts
results.append(adfuller_test_df(test_zip,index=['Original']))

## Loop through decomp dict, 
for trend, ts_ in decomp_dict.items():
    # Fill any missing values, get adfuller result
    ts_ = ts_.fillna(0)
    res = adfuller_test_df(ts_,index=trend)
    results.append(res)

    
    ## Append res to decomp_stationary

## make into a df
res_df = pd.concat(results)
res_df

# ❌ TODO: manually code, then  functionize

## ACF/PACF Check (Tr)

In [ ]:
def plot_acf_pacf(ts,figsize=(9,6),lags=52,suptitle=None,sup_y = 1.01):
    """Plot pacf and acf using statsmodels"""
    fig,axes=plt.subplots(nrows=2,figsize=figsize)
    
    tsa.graphics.plot_acf(ts,ax=axes[0],lags=lags);
    tsa.graphics.plot_pacf(ts,ax=axes[1],lags=lags);
    
    ## Add grid
    [ax.grid(axis='x',which='both') for ax in axes]
    
    if suptitle is not None:
        fig.suptitle(suptitle,y=sup_y,fontweight='bold',fontsize=15)
        
    fig.tight_layout()
    return fig,axes

In [ ]:
## Plot ACF/PACF train, train_diff

## Establishing Stationarity

---

> Loop through each of the stationarity methods and plot results
>
>
> Post plotting, add comments/review
>
>
> ***Remember:*** Auto-Arima/SARIMAX uses orignal data - **purpose of this exploration** is for EDA/determining the "d" value for testing.

---

In [ ]:
tz_diff = test_zip.diff().dropna()
tz_diff.plot()
adfuller_test_df(tz_diff)

In [ ]:
## Log Transform, plot and get adfuller test
tz_log = np.log(test_zip)
tz_log.plot()
adfuller_test_df(tz_log)

In [ ]:
## Subtract Rolling mean
tz_rm = test_zip - test_zip.rolling(window=4).mean()
tz_rm.dropna(inplace=True)
tz_rm.plot()
adfuller_test_df(tz_rm)

In [ ]:
tz_ewm = test_zip-test_zip.ewm(4).mean()
tz_ewm.dropna(inplace=True)
tz_ewm.plot()
adfuller_test_df(tz_ewm)

### Seasonal Decomp (Tr)

In [ ]:
## Checking for seasonality
decomp = tsa.seasonal_decompose(train)
decomp.plot();

In [ ]:
decomp = seasonal_decompose(test_zip)
decomp.plot();

In [ ]:
decomp.seasonal

In [ ]:
## Save seasonal/trend/resid in a dictionary.

decomp_dict = {'seasonal': decomp.seasonal,
              "trend": decomp.trend,
              'residuals': decomp.resid}

decomp_dict

In [ ]:
## Make a list of adfuller results to append
results = []
## Save results of orig ts
results.append(adfuller_test_df(test_zip,index=['Original']))

## Loop through decomp dict, 
for trend, ts_ in decomp_dict.items():
    # Fill any missing values, get adfuller result
    ts_ = ts_.fillna(0)
    res = adfuller_test_df(ts_,index=trend)
    results.append(res)

    
    ## Append res to decomp_stationary

## make into a df
res_df = pd.concat(results)
res_df

In [ ]:
## Pldot decomp again for convenient comparison
decomp.plot();

## Auto-Arima (Tr)

In [ ]:
test_zip.plot()

In [ ]:
## Use auto_arima 
auto_model = pmd.auto_arima(test_zip.loc['2008':],start_p=0,start_q=0,d=1,
                            max_p=3,max_q=3,
                            max_P=3,max_Q=3,
                            start_P=0,start_Q=0,
                            m=12,
                            verbose=2)

In [ ]:
display(auto_model.summary())
auto_model.plot_diagnostics();
plt.tight_layout()

# ❌ TODO: manually code, then  functionize

In [ ]:
def diagnose_model(model):
    """Takes a fit statsmodels model and displays the .summary 
    and plots the built-in plot.diagnostics()"""
    display(model.summary())
    model.plot_diagnostics()
    plt.tight_layout()

## Fit Best Model & Eval (Tr)

In [2]:
def calc_plot_best_model(train,start_p=0,max_p=5,start_q=0,max_q=5,d=1,m=52,
                         start_P=0,start_Q=0, verbose = True):
    
    auto_model = pmd.auto_arima(train,start_p=start_p,max_p=max_p,
                           start_q=start_q,max_q=max_q,d=d,m=m,
                           start_P=start_P,start_Q=start_Q,verbose=verbose)
    
    display(auto_model.summary())
    
    best_model = tsa.SARIMAX(train,order=auto_model.order,
                             seasonal_order = auto_model.seasonal_order,
                             enforce_invertibility=False).fit()
    
    ## Display Summary + Diagnostics
    display(best_model.summary())
    best_model.plot_diagnostics();
    plt.tight_layout()
    
    return auto_model, best_model

## Forecasting

---

> Save `conf_int`, `predicted_mean` - 4cDF
>
>
> Plot Tr, Te, 4cDF

---

# ❌ TODO: manually code, then  functionize

In [ ]:
def forecast_and_plot(train, test, final_ts = None, model, last_n_lags=52,
                      x_label, y_label, figsize=(10,4)):

    ## Get forecast
    forecast = model.get_forecast(steps=len(test))

    ## Save forecasted mean, upper/lower CI as DF
    forecast_df = forecast.conf_int()
    forecast_df.columns = ['Lower CI','Upper CI']
    forecast_df['Forecast'] = forecast.predicted_mean

    # Plotting timeseries data
    fig,ax = plt.subplots(figsize=figsize)
    
    last_n_lags=last_n_lags
    
    if final_ts is None:
        train.iloc[-last_n_lags:].plot(label='Training Data')
        test.plot(label='Test Data')
    else:
        ts.iloc[-last_n_lags:].plot(label='Training Data')
        ax.axvline(ts.index[-1],ls=':')

    ## Plotting forecast and CI
    forecast_df['Forecast'].plot(ax=ax,label='Forecast')
    ax.fill_between(forecast_df.index,
                    forecast_df['Lower CI'], 
                    forecast_df['Upper CI'],color='g',alpha=0.3)

    ax.set(xlabel=x_label)
    ax.set(ylabel=y_label)
    ax.legend()
    plt.show();
    
    return forecast_df

In [ ]:
def get_df_from_pred(forecast_or_pred,forecast_label='Forecast'):
    """Takes a PredictionResultsWrapper from statsmodels
    extracts the confidence intervals and predicted mean and returns in a df"""
    forecast_df = forecast_or_pred.conf_int()
    forecast_df.columns = ['Lower CI','Upper CI']
    forecast_df[forecast_label] = forecast_or_pred.predicted_mean
    return forecast_df

def plot_forecast_from_df(forecast_df,ts_diff=None,orig_label='True Data',
                          forecast_label='Forecast',
                          last_n_lags=52,figsize=(10,4)):
    """Takes a forecast_df from get_df_from_pred and optionally 
    the training/original time series.
    
    Plots the original ts, the predicted mean and the 
    confidence invtervals (using fill between)"""
    fig,ax = plt.subplots(figsize=figsize)

    if ts_diff is not None:
        ts_diff.iloc[-last_n_lags:].plot(label='True Data')
        
   
    forecast_df['Forecast'].plot(ax=ax,label=forecast_label)
    ax.fill_between(forecast_df.index,
                    forecast_df['Lower CI'], 
                    forecast_df['Upper CI'],color='g',alpha=0.3)
    ax.legend()
    ax.set(title=f'Forecasted {ts_diff.name}')
    return fig,ax\

## Run on Full Dataset for Final Results

In [ ]:
## If happy with the model's test perforamance, retrain on entire ts and forecast into future
## Fit a final model and evaluate
final_model = tsa.SARIMAX(ts,order=auto_model.order,
                seasonal_order = auto_model.seasonal_order,
                enforce_invertibility=False).fit()


## Display Summary + Diagnostics
display(final_model.summary())
final_model.plot_diagnostics();
plt.tight_layout()

In [ ]:
## Get forecast 
forecast = final_model.get_forecast(steps=len(test))

## save forecasted mean and upper/lower ci as df
forecast_df = forecast.conf_int()
forecast_df.columns = ['Lower CI','Upper CI']
forecast_df['Forecast'] = forecast.predicted_mean

## Plot
last_n_lags=52

fig,ax = plt.subplots(figsize=(10,4))

                      
# Plotting Training and test data
ts.iloc[-last_n_lags:].plot(label='Training Data')
ax.axvline(ts.index[-1],ls=':')
# test.plot(label='Test Data')

## Plotting Forefcast and CI
forecast_df['Forecast'].plot(ax=ax,label='Forecast')
ax.fill_between(forecast_df.index,
                forecast_df['Lower CI'], 
                forecast_df['Upper CI'],color='g',alpha=0.3)

ax.set(ylabel='Crime Count')
ax.legend()

## Interpreting Results

---
> **Reminder:** the goal is to determine the top 5 best zip codes for investment.
> * *Need to define "best investment" - ROI, risk, ???*
>
>
>
---

### Display final forecasted results

## Create Dict 2 Store Results

---

> [See this link](https://www.youtube.com/watch?v=atUym8mOnNc&list=PLFknVelSJiSxSwXifV_ysDg50fzbuTzVt&index=59)

---

In [5]:
raise ValueError('Old notebook code - DO NOT RUN')

ValueError: Old notebook code - DO NOT RUN

# Mod 4 Project - Starter Notebook

This notebook has been provided to you so that you can make use of the following starter code to help with the trickier parts of preprocessing the Zillow dataset. 

The notebook contains a rough outline the general order you'll likely want to take in this project. You'll notice that most of the areas are left blank. This is so that it's more obvious exactly when you should make use of the starter code provided for preprocessing. 

**_NOTE:_** The number of empty cells are not meant to infer how much or how little code should be involved in any given step--we've just provided a few for your convenience. Add, delete, and change things around in this notebook as needed!

## Some Notes Before Starting

This project will be one of the more challenging projects you complete in this program. This is because working with Time Series data is a bit different than working with regular datasets. In order to make this a bit less frustrating and help you understand what you need to do (and when you need to do it), we'll quickly review the dataset formats that you'll encounter in this project. 

## Wide Format vs Long Format

If you take a look at the format of the data in `zillow_data.csv`, you'll notice that the actual Time Series values are stored as separate columns. Here's a sample: 

<img src='https://raw.githubusercontent.com/learn-co-students/dsc-mod-4-project-seattle-ds-102819/master/images/df_head.png'>

You'll notice that the first seven columns look like any other dataset you're used to working with. However, column 8 refers to the median housing sales values for April 1996, column 9 for May 1996, and so on. This This is called **_Wide Format_**, and it makes the dataframe intuitive and easy to read. However, there are problems with this format when it comes to actually learning from the data, because the data only makes sense if you know the name of the column that the data can be found it. Since column names are metadata, our algorithms will miss out on what dates each value is for. This means that before we pass this data to our ARIMA model, we'll need to reshape our dataset to **_Long Format_**. Reshaped into long format, the dataframe above would now look like:

<img src='https://raw.githubusercontent.com/learn-co-students/dsc-mod-4-project-seattle-ds-102819/master/images/melted1.png'>

There are now many more rows in this dataset--one for each unique time and zipcode combination in the data! Once our dataset is in this format, we'll be able to train an ARIMA model on it. The method used to convert from Wide to Long is `pd.melt()`, and it is common to refer to our dataset as 'melted' after the transition to denote that it is in long format.

## Helper Functions Provided

Melting a dataset can be tricky if you've never done it before, so you'll see that we have provided a sample function, `melt_data()`, to help you with this step below. Also provided is:

* `get_datetimes()`, a function to deal with converting the column values for datetimes as a pandas series of datetime objects
* Some good parameters for matplotlib to help make your visualizations more readable. 

Good luck!

# Reading Data

In [ ]:
# # import warnings
# # warnings.filterwarnings('ignore')

# ## Data Handling
# import pandas as pd
# import numpy as np

# ## Visualizations
# import matplotlib as mpl
# import matplotlib.pyplot as plt
# import seaborn as sns

# import statsmodels
# import statsmodels.tsa.api as tsa
# from statsmodels.tsa.seasonal import seasonal_decompose

# import pmdarima as pmd

# ## Settings
# %matplotlib inline
# plt.rcParams['figure.figsize']=(12,6)
# plt.style.use('seaborn-talk')
# pd.set_option('display.max_columns', None)
# pd.set_option('display.float_format', lambda x: f'{x:,.2f}')
# pd.set_option('max_rows', 100)

# from bmc_functions import eda

In [ ]:
# %load_ext autoreload
# %autoreload 2

In [ ]:
# ## ImportingReading in-Dta
# source = '../data/zillow_data.csv'
# data = pd.read_csv(source)
# data.head(5)

In [3]:
# data.info()

# Filtering Data

---

> The dataset is much larger than I need for my purposes, so I will determine a smaller regional subset for analysis.
>
>
> As I am from Pittsburgh, PA, I will select that region for my models.

---

In [ ]:
# ## Selecting Pittsburgh Metro Area

# pitt_df = data[data['City'] == 'Pittsburgh']
# pitt_df

In [ ]:
# ## Examining Statistics for the Pittsburgh Metro area
# eda.report_df(pitt_df).T

In [ ]:
# ## Inspecting overall data for CA - transposed and dropping RegionID, SizeRank

# pitt_zips = pitt_df.pivot_table(index= 'RegionName').T[:-2]
# pitt_zips

# Data Preprocessing

In [ ]:
# def get_datetimes(df):
#     """
#     Takes a dataframe:
#     returns only those column names that can be converted into datetime objects 
#     as datetime objects.
#     NOTE number of returned columns may not match total number of columns in passed dataframe
#     """
    
#     return pd.to_datetime(df.columns.values[7:], format='%Y-%m')

In [ ]:
# pitt_df.columns.values[7:] = get_datetimes(pitt_df)
# pitt_df

# Melting DataFrame

In [ ]:
# def melt_data(df):
#     """
#     Takes the zillow_data dataset in wide form or a subset of the zillow_dataset.  
#     Returns a long-form datetime dataframe with the datetime column names
#     as the index and the values as the 'values' column.
    
#     If more than one row is passes in the wide-form dataset, the values column
#     will be the mean of the values from the datetime columns in all of the rows.
#     """
    
#     melted = pd.melt(df, id_vars=['RegionName', 'RegionID', 'SizeRank',
#                                   'City', 'State', 'Metro', 'CountyName'],
#                      var_name='time')
#     melted['time'] = pd.to_datetime(melted['time'], infer_datetime_format=True)
#     melted = melted.dropna(subset=['value'])
#     return melted#.groupby('time').aggregate({'value':'mean'})

In [ ]:
# pitt_melted = melt_data(pitt_df)
# pitt_melted

## Old Code - Pivot Table

In [ ]:
# ## Inspecting overall data for Pittsburgh

# pitt_df.pivot_table(index= 'RegionName')

In [ ]:
# ## Inspecting overall data for CA - transposed and dropping RegionID, SizeRank

# pitt_zips = pitt_df.pivot_table(index= 'RegionName').T[:-2]
# pitt_zips

# Step 3: EDA and Visualization

In [ ]:
# font = {'family' : 'normal',
#         'weight' : 'bold',
#         'size'   : 22}

# mpl.rc('font', **font)

# # NOTE: if you visualizations are too cluttered to read, try calling 'plt.gcf().autofmt_xdate()'!

In [ ]:
# ## Generating initial statistical overview
# report = eda.report_df(pitt_df)
# report

In [ ]:
## Inspecting only zip codes with missing columns
# report[report['null_sum'] >0]

In [ ]:
# ## Selecting zipcode with largest number of entries
# most_freq_zip = report['count'].sort_values(ascending=False)[:1]
# most_freq_zip

In [ ]:
# ## Checking for missing values pre-visualizing
# pitt_zips[15243].isna().sum()

In [ ]:
# test_zip = pitt_zips[15243]

In [ ]:
# ## Initial visualization of one zipcode
# test_zip.plot();

# ❌ FIX/REMOVE JMI FUNCTION

In [ ]:
# ## Lab Function
# # from statsmodels.tsa.stattools import adfuller

# def adfuller_test_df(ts,index=['AD Fuller Results']):
#     """Returns the AD Fuller Test Results and p-values for the null hypothesis
#     that there the data is non-stationary (that there is a unit root in the data)"""
    
#     df_res = tsa.stattools.adfuller(ts)
    
#     names = ['Test Statistic','p-value','#Lags Used','# of Observations Used']
#     res  = dict(zip(names,df_res[:4]))
    
#     res['p<.05'] = res['p-value']<.05
#     res['Stationary?'] = res['p<.05']
    
#     if isinstance(index,str):
#         index = [index]
#     res_df = pd.DataFrame(res,index=index)
#     res_df = res_df[['Test Statistic','#Lags Used',
#                      '# of Observations Used','p-value','p<.05',
#                     'Stationary?']]
#     return res_df



# def stationarity_check(TS,window=8,plot=True,index=['AD Fuller Results']):
#     """Adapted from https://github.com/learn-co-curriculum/dsc-removing-trends-lab/tree/solution"""
    
#     # Calculate rolling statistics
#     roll_mean = TS.rolling(window=window, center=False).mean()
#     roll_std = TS.rolling(window=window, center=False).std()
    
#     # Perform the Dickey Fuller Test
#     dftest = adfuller_test_df(TS,index=index)
    
#     if plot:
#         # Plot rolling statistics:
#         fig = plt.figure(figsize=(12,6))
#         plt.plot(TS, color='blue',label=f'Original (freq={TS.index.freq})')
#         plt.plot(roll_mean, color='red', label=f'Rolling Mean (window={window})')
#         plt.plot(roll_std, color='black', label = f'Rolling Std (window={window})')
#         plt.legend(loc='best')
#         plt.title('Rolling Mean & Standard Deviation')
#         display(dftest)
#         plt.show(block=False)
        
#     return dftest
    

In [ ]:
# from statsmodels.tsa.stattools import adfuller
# results = stationarity_check(test_zip)

In [ ]:
# tz_diff = test_zip.diff().dropna()
# tz_diff.plot()
# adfuller_test_df(tz_diff)

In [ ]:
# ## Log Transform, plot and get adfuller test
# tz_log = np.log(test_zip)
# tz_log.plot()
# adfuller_test_df(tz_log)

In [ ]:
# ## Subtract Rolling mean
# tz_rm = test_zip - test_zip.rolling(window=4).mean()
# tz_rm.dropna(inplace=True)
# tz_rm.plot()
# adfuller_test_df(tz_rm)

In [ ]:
# tz_ewm = test_zip-test_zip.ewm(4).mean()
# tz_ewm.dropna(inplace=True)
# tz_ewm.plot()
# adfuller_test_df(tz_ewm)

In [ ]:
# decomp = seasonal_decompose(test_zip)
# decomp.plot();

In [ ]:
# decomp.seasonal

In [ ]:
# ## Save seasonal/trend/resid in a dictionary.

# decomp_dict = {'seasonal': decomp.seasonal,
#               "trend": decomp.trend,
#               'residuals': decomp.resid}

# decomp_dict

In [ ]:
# ## Make a list of adfuller results to append
# results = []
# ## Save results of orig ts
# results.append(adfuller_test_df(test_zip,index=['Original']))

# ## Loop through decomp dict, 
# for trend, ts_ in decomp_dict.items():
#     # Fill any missing values, get adfuller result
#     ts_ = ts_.fillna(0)
#     res = adfuller_test_df(ts_,index=trend)
#     results.append(res)

    
#     ## Append res to decomp_stationary

# ## make into a df
# res_df = pd.concat(results)
# res_df

In [ ]:
# ## Pldot decomp again for convenient comparison
# decomp.plot();

# Step 5: ARIMA Modeling

In [ ]:
# test_zip.plot()

In [ ]:
# ## Use auto_arima 
# auto_model = pmd.auto_arima(test_zip.loc['2008':],start_p=0,start_q=0,d=1,
#                             max_p=3,max_q=3,
#                             max_P=3,max_Q=3,
#                             start_P=0,start_Q=0,
#                             m=12,
#                             verbose=2)

In [ ]:
# display(auto_model.summary())
# auto_model.plot_diagnostics();
# plt.tight_layout()

In [ ]:
# tz_diff = test_zip.diff().dropna()
# tz_diff

In [ ]:
# adfuller_test_df(tz_diff)

In [ ]:
# am_diff = pmd.auto_arima(test_zip,start_p=0,start_q=0, d=1,
#                             max_p=4,max_q=3,
#                             max_P=3,max_Q=2,
#                             start_P=0,start_Q=0,
#                             m=12,
#                             verbose=2)

In [ ]:
# display(am_diff.summary())
# am_diff.plot_diagnostics();
# plt.tight_layout()

In [ ]:
# ### From SARIMA Models Lab
# import itertools
# from tqdm.notebook import trange
# # Define the p, d and q parameters to take any value between 0 and 2
# ps = list(range(0,4))
# ds = list(range(0,2))
# qs = list(range(0,3))

# # Generate all different combinations of p, q and q triplets
# pdq_list = list(itertools.product(ps,ds, qs))
# pdq_list

In [ ]:
# ## Loop through pdq_list, make an ARIMA model
# # save p,d,q and aic to a model_aic list
# model_aics= [['p','d','q','aic']]

# ## Make Results into a df and sort by aic
# for i in trange(len(pdq_list)):
#     p,d,q = pdq_list[i]
#     model = tsa.arima.ARIMA(ts,order=(p,d,q),enforce_invertibility=False).fit()
#     model_aics.append([p,d,q,model.aic])
#     print(f'For ({p},{d},{q}), aic = {model.aic:.3f}')

# results = pd.DataFrame(model_aics[1:],columns=model_aics[0]).sort_values('aic')
# results